# Text Normalization (Chinese)

- `text_normalizer_zh.py`
- Including functions for:
    - word-seg chinese texts
    - clean up texts by removing duplicate spaces and line breaks
    - remove incompatible weird characters

In [1]:
import unicodedata
import re
#from nltk.corpus import wordnet
#import collections
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup
import requests
import pandas as pd
import text_normalizer_zh as tnz

## Normalization Functions

In [2]:
# %load text_normalizer_zh.py
"""

Notes
-----

    These functions are based on the text normalization functions 
    provided in Text Analytics with Python 2ed.

"""

import unicodedata
import re
# from nltk.tokenize.toktok import ToktokTokenizer
import pandas as pd
import jieba

## Initialize Trad Chinese dictionary
jieba.set_dictionary('../../../RepositoryData/data/jiaba/dict.txt.jiebatw.txt')


## Normalize unicode characters
def remove_weird_chars(text):
    #     ```
    #     (NFKD) will apply the compatibility decomposition, i.e.
    #     replace all compatibility characters with their equivalents.
    #     ```
    text = unicodedata.normalize('NFKD', text).encode('utf-8',
                                                      'ignore').decode(
                                                          'utf-8', 'ignore')
    return text


## Remove extra linebreaks
def remove_extra_linebreaks(text):
    lines = text.split(r'\n+')
    return '\n'.join(
        [re.sub(r'[\s]+', ' ', l).strip() for l in lines if len(l) != 0])


## Remove extra medial/trailing/leading spaces
def remove_extra_spaces(text):
    return re.sub("\\s+", " ", text).strip()


## Seg the text into words
def seg(text):
    text_seg = jieba.cut(text)
    out = ' '.join(text_seg)
    return out


## Remove punctuation/symbols
def remove_symbols(text):
    """
    
    Unicode 6.0 has 7 character categories, and each category has subcategories:

    Letter (L): lowercase (Ll), modifier (Lm), titlecase (Lt), uppercase (Lu), other (Lo)
    Mark (M): spacing combining (Mc), enclosing (Me), non-spacing (Mn)
    Number (N): decimal digit (Nd), letter (Nl), other (No)
    Punctuation (P): connector (Pc), dash (Pd), initial quote (Pi), final quote (Pf), open (Ps), close (Pe), other (Po)
    Symbol (S): currency (Sc), modifier (Sk), math (Sm), other (So)
    Separator (Z): line (Zl), paragraph (Zp), space (Zs)
    Other (C): control (Cc), format (Cf), not assigned (Cn), private use (Co), surrogate (Cs)
    
    
    There are 3 ranges reserved for private use (Co subcategory): 
    U+E000—U+F8FF (6,400 code points), U+F0000—U+FFFFD (65,534) and U+100000—U+10FFFD (65,534). 
    Surrogates (Cs subcategory) use the range U+D800—U+DFFF (2,048 code points).
    
    
    """

    ## Brute-force version: list all possible unicode ranges, but this list is not complete.
    #   text = re.sub('[\u0021-\u002f\u003a-\u0040\u005b-\u0060\u007b-\u007e\u00a1-\u00bf\u2000-\u206f\u2013-\u204a\u20a0-\u20bf\u2100-\u214f\u2150-\u218b\u2190-\u21ff\u2200-\u22ff\u2300-\u23ff\u2460-\u24ff\u2500-\u257f\u2580-\u259f\u25a0-\u25ff\u2600-\u26ff\u2e00-\u2e7f\u3000-\u303f\ufe50-\ufe6f\ufe30-\ufe4f\ufe10-\ufe1f\uff00-\uffef─◆╱]+','',text)

    text = ''.join(ch for ch in text
                   if unicodedata.category(ch)[0] not in ['P', 'S'])
    return text


## Remove numbers
def remove_numbers(text):
    return re.sub('\\d+', "", text)


## Remove alphabets
def remove_alphabets(text):
    return re.sub('[a-zA-Z]+', '', text)


## Combine every step
def normalize_corpus(corpus,
                     is_remove_extra_linebreaks=True,
                     is_remove_weird_chars=True,
                     is_seg=True,
                     is_remove_symbols=True,
                     is_remove_numbers=True,
                     is_remove_alphabets=True):

    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:

        if is_remove_extra_linebreaks:
            doc = remove_extra_linebreaks(doc)

        if is_remove_weird_chars:
            doc = remove_weird_chars(doc)

        if is_seg:
            doc = seg(doc)

        if is_remove_symbols:
            doc = remove_symbols(doc)

        if is_remove_alphabets:
            doc = remove_alphabets(doc)

        if is_remove_numbers:
            doc = remove_numbers(doc)

        normalized_corpus.append(remove_extra_spaces(doc))

    return normalized_corpus

## Extract an Article

- Grab the first article from Google news for demonstration

In [3]:
url = 'https://news.google.com/topics/CAAqJQgKIh9DQkFTRVFvSUwyMHZNRFptTXpJU0JYcG9MVlJYS0FBUAE?hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
r = requests.get(url)
web_content = r.text
soup = BeautifulSoup(web_content, 'lxml')
title = soup.find_all('a', class_='DY5T1d')
first_art_link = title[0]['href'].replace('.', 'https://news.google.com', 1)

#print(first_art_link)
art_request = requests.get(first_art_link)
art_request.encoding = 'utf8'
soup_art = BeautifulSoup(art_request.text, 'lxml')

art_content = soup_art.find_all('p')
art_texts = [p.text for p in art_content]
print(art_texts)

['新北市警新店分局今天凌晨1時許接獲報案，一名社區警衛在警衛亭聽到一聲巨響，外出查看，發現一名男子倒臥在路上；警方獲報趕往調查，發現是18歲的王姓男子，初步研判是從住家12樓頂樓墜落，現場已無呼吸心跳，經送醫急救，仍於凌晨2時許宣告不治。經了解，王目前就讀高雄海軍官校。', '\n', '\r\n據了解，18歲王姓男子住在新北市新店區永安街一處社區大樓，平時住校，上周四因雙十國慶返家過連假，昨天晚間10時原本應該要收假回到學校；沒想到今天凌晨1時許，卻從住家社區大樓12樓頂樓墜落至1樓馬路，頭部受到重擊當場沒有呼吸心跳，家屬聽到聲響外出查看，發現是王，當場痛哭失聲。', '\n', '\r\n家屬表示死者回家時並沒有異狀，連假期間還與家人有說有笑，甚至還去找表哥、表姊烤肉聊天，很難相信王會做傻事。', '\n', '\r\n警方趕往後，封鎖現場採證，在頂樓發現王姓男子的黑色行李袋，經鑑識人員初步勘察，現場無打鬥跡象，初步排除他殺嫌疑，也沒有發現遺書，研判為輕生可能性大，詳細原因仍有待警方進一步釐清，目前已請家屬至派出所製作筆錄調查中。', '\n', '\n 自殺，不能解決難題；求助，才是最好的路。求救請打1995 ( 要救救我 ) ', '\n', '\n新北市警新店分局今天凌晨1時許接獲報案，一名社區警衛在警衛亭聽到一聲巨響，外出查看，發現一名男子倒臥在路上。記者袁志豪／翻攝', '\n新北市警新店分局今天凌晨1時許接獲報案，一名社區警衛在警衛亭聽到一聲巨響，外出查看，發現一名男子倒臥在路上。記者袁志豪／翻攝新北市警新店分局今天凌晨1時許接獲報案，一名社區警衛在警衛亭聽到一聲巨響，外出查看，發現一名男子倒臥在路上。記者袁志豪／翻攝', '\n                    新北市警新店分局今天凌晨1時許接獲報案，一名社區警衛在警衛亭聽到一聲巨響，外出查看，發現一名男子倒臥在路上；警方獲報趕往...                  ', '\n                    王姓海軍官校學生今晨在新北市新店區安和路住家墜樓身亡，海軍上午證實該名學生確就讀海軍官校一年級，目前官校正與家屬取得聯繫...                  ', '\n                    高雄市警仁武分局澄觀派出所所長薛冠洲今晨執行釣蝦場勤務，疑似

- Normalized results:

In [4]:
tnz.normalize_corpus([' '.join(art_texts)])

Building prefix dict from /Users/Alvin/GoogleDrive/_MySyncDrive/RepositoryData/data/jiaba/dict.txt.jiebatw.txt ...


Loading model from cache /var/folders/n7/ltpzwx813c599nfxfb94s_640000gn/T/jieba.u1b52b47246a0f2e6497af6bbe107adac.cache


Loading model cost 0.531 seconds.


Prefix dict has been built successfully.


['新北 市警 新店 分局 今天 凌晨 時許 接獲 報案 一名 社區 警衛 在 警衛亭 聽到 一聲 巨響 外出 查看 發現 一名 男子 倒臥 在 路 上 警方 獲報 趕往 調查 發現 是 歲 的 王姓 男子 初步 研判 是 從 住家 樓 頂樓 墜落 現場 已 無 呼吸 心跳 經 送醫 急救 仍 於 凌晨 時許 宣告 不治 經 了解 王 目前 就讀 高雄 海軍 官校 據了解 歲 王姓 男子 住 在 新 北市 新店區 永安街 一處 社區 大樓 平時 住校 上 周四 因 雙十國慶 返家過 連假 昨天 晚間 時 原本 應該 要 收假 回到 學校 沒想到 今天 凌晨 時許 卻 從 住家 社區 大樓 樓 頂樓 墜落 至 樓 馬路 頭部 受到 重擊 當場 沒有 呼吸 心跳 家屬 聽到 聲響 外出 查看 發現 是 王 當場 痛哭失聲 家屬 表示 死者 回家 時並 沒有 異狀 連假 期間 還與 家人 有說有笑 甚至 還去 找 表哥 表姊 烤肉 聊天 很 難 相信 王會 做 傻事 警方 趕往 後 封鎖 現場 採證 在 頂樓 發現 王姓 男子 的 黑色 行李袋 經 鑑識 人員 初步 勘察 現場 無 打鬥 跡象 初步 排除 他殺 嫌疑 也 沒有 發現 遺書 研判 為 輕生 可能性 大 詳細 原因 仍 有待 警方 進一步 釐清 目前 已 請 家屬 至 派出所 製作 筆錄 調查 中 自殺 不能 解決 難題 求助 才 是 最好 的 路 求救 請 打 要 救救 我 新北 市警 新店 分局 今天 凌晨 時許 接獲 報案 一名 社區 警衛 在 警衛亭 聽到 一聲 巨響 外出 查看 發現 一名 男子 倒臥 在 路 上 記者 袁 志豪 翻攝 新北 市警 新店 分局 今天 凌晨 時許 接獲 報案 一名 社區 警衛 在 警衛亭 聽到 一聲 巨響 外出 查看 發現 一名 男子 倒臥 在 路 上 記者 袁 志豪 翻攝 新北 市警 新店 分局 今天 凌晨 時許 接獲 報案 一名 社區 警衛 在 警衛亭 聽到 一聲 巨響 外出 查看 發現 一名 男子 倒臥 在 路 上 記者 袁 志豪 翻攝 新北 市警 新店 分局 今天 凌晨 時許 接獲 報案 一名 社區 警衛 在 警衛亭 聽到 一聲 巨響 外出 查看 發現 一名 男子 倒臥 在 路 上 警方 獲報 趕往 王姓 海軍 官校 學生 今晨 在 新 北市 新店區 安和路 住